In [1]:
"""
Feature: urinalysis_wbc
Extract max urinalysis WBC from labevents
"""
import pandas as pd
import os
import re
from config_paths import *
from utils import drop_columns

In [2]:
# Read dataset
df = pd.read_csv(dataset_path)
initial_row_count = len(df)
df["admittime"] = pd.to_datetime(df["admittime"])
df["dischtime"] = pd.to_datetime(df["dischtime"])

# Load labevents
labevents = pd.read_csv(
    os.path.join(hosp_path, "labevents.csv"),
    usecols=["subject_id", "hadm_id", "itemid", "charttime", "valuenum"]
)
labevents["charttime"] = pd.to_datetime(labevents["charttime"])

# Urinalysis WBC item IDs
urinalysis_wbc_ids = [51516]

# Filter and merge
urine_wbc = labevents[labevents["itemid"].isin(urinalysis_wbc_ids)].copy()
urine_wbc = urine_wbc.merge(
    df[["subject_id", "hadm_id", "admittime", "dischtime"]],
    on=["subject_id", "hadm_id"],
    how="left"
)

# Restrict to admission window
urine_wbc = urine_wbc[
    (urine_wbc["charttime"] >= urine_wbc["admittime"]) &
    (urine_wbc["charttime"] <= urine_wbc["dischtime"])
]

# Max per admission
urine_wbc_max = (
    urine_wbc.groupby(["subject_id", "hadm_id"])["valuenum"]
    .max()
    .reset_index()
    .rename(columns={"valuenum": "urinalysis_wbc"})
)

# Merge
df = df.merge(urine_wbc_max, on=["subject_id", "hadm_id"], how="left")

# Save
df.to_csv(dataset_path, index=False)

# Diagnostic output
print(f"\nTotal rows: {len(df)}")
print(f"Expected total: {initial_row_count}")
if len(df) == initial_row_count:
    print(f"✓ Row count matches expected total ({initial_row_count})")
else:
    print(f"⚠ Row count mismatch! Expected {initial_row_count}, got {len(df)} (difference: {len(df) - initial_row_count})")
print(f"Dataset shape: {df.shape}")
print(f"Feature 'urinalysis_wbc' added. Non-null count: {df['urinalysis_wbc'].notna().sum()}")
print(f"Dataset shape: {df.shape}")




Total rows: 158020
Expected total: 158020
✓ Row count matches expected total (158020)
Dataset shape: (158020, 77)
Feature 'urinalysis_wbc' added. Non-null count: 47646
Dataset shape: (158020, 77)


In [3]:
df.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race', 'catheter_procedures',
       'catheter_procedures_ir', 'catheter_used_in_procedures_ir',
       'catheter_insertion_date_ir', 'catheter_removal_date_ir',
       'catheter_outputevents', 'catheter_used_in_output_events',
       'datetimeevents', 'catheter_used_in_datetime_events',
       'insertion_date_from_datetimeevents',
       'removal_date_from_datetimeevents', 'catheter_procedure_events',
       'catheter_used_in_procedure_events', 'catheter_size_from_chartevents',
       'index', 'catheter_clinical_notes', 'catheter_used_in_clinical_notes',
       'catheter_present', 'final_cauti_flag', 'final_insertion_date',
       'final_removal_date', 'catheter_duration_days', 'BMI_in_admission',
       'BMI_computed', 'BMI_last_year', 'BMI', 

In [5]:
# cols_to_drop = [
#  "urinalysis_wbc"
# ]
# df = drop_columns(df, cols_to_drop)
# df.to_csv(dataset_path, index=False)